In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math
import random

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

In [2]:
def readAndShowImage(image):
    im = io.imread(image)
    io.imshow(im)
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Strand':'strand.jpeg',
        'Mandrill':'mandrill.png',
        'Trump':'trump.jpg',
        'Gruppe':'gruppe.jpg',
        'RGB':"rgb.png",
        'Dress':'TheDress.png'
        },
    description='Bilde: ',
    disabled=False,
)

interactive(readAndShowImage, image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…

In [3]:
def distance(pixel1, pixel2):
    return ((pixel1[0] - pixel2[0])**2 + (pixel1[1] - pixel2[1])**2 + (pixel1[2] - pixel2[2])**2)**0.5

def segment(im, numSegments):  
    im = util.img_as_float64(im)
    im = im[:,:,:3]
    width, height, depth = im.shape
    
    segments = []
    
    #Plukke ut N tilfeldige pixler som skal brukes som start punkter
    for x in range(numSegments):
        segment = {}
        segment["pixel"] = im[random.randint(0,width-1),random.randint(0,height-1)]
        segment["data"] = {"sum":[0,0,0],"num":0}
        segments.append(segment)
    
    while(True):
        #Finne gjennomsnittsfargen for hvert punkt
        for x in range(width):
            for y in range(height):
                closest = 0
                closestDistance = math.inf
                for segment in segments:
                    dist = distance(im[x,y],segment["pixel"])
                    if( dist < closestDistance):
                        closestDistance = dist
                        closest = segment
                closest["data"]["sum"] += im[x,y]
                closest["data"]["num"] += 1

        #Flytter punktet til gjennomsnittetsfargen til de nærmeste pixlene
        movement = 0
        for segment in segments:
            segment["data"]["prevPixel"] = segment["pixel"]
            if(segment["data"]["num"] != 0):
                segment["pixel"] = segment["data"]["sum"] / segment["data"]["num"]
            else:
                segment["pixel"] = [0,0,0]
            movement += distance(segment["data"]["prevPixel"],segment["pixel"])
            segment["data"]["sum"] = [0,0,0]
            segment["data"]["num"] = 0
        
        print(movement/numSegments)
        if (movement/numSegments < 0.001):
            break
    
    #Sette hver pixel til gjennomsnittet til det nærmeste punktet
    for x in range(width):
        for y in range(height):
            closest = 0
            closestDistance = math.inf
            for segment in segments:
                dist = distance(im[x,y],segment["pixel"])
                if( dist < closestDistance):
                    closestDistance = dist
                    closest = segment
            im[x,y] = closest["pixel"]
            
    
    return im

def update(image, numSegments):
    im = io.imread(image)
    
    try:
        segmented = io.imread("Generated/" + image.split(".")[0] + str(numSegments) + ".png")
    except:
        print("Image not generated, calculating new")
        segmented = segment(im,numSegments)
        
    io.imsave("Generated/" + image.split(".")[0] + str(numSegments) + ".png",segmented)
    io.imshow(segmented)
    io.show()
    
    
interactive(update, image=inputImage, numSegments=widgets.IntSlider(value = 4,min = 2,max = 32,description='Number of segments: ',continuous_update = False)) 

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…